In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import datetime
import re
import cx_Oracle
import numpy as np
import datetime

# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()

username = 'ebs_apps'
password = 'ebs_apps'
host = 'csdwddb.cswg.com' #EBSD2
port = '1521'
service_name = 'DWD_USR' #EBSD2

# Create a connection
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
oracle_con = cx_Oracle.connect(username, password, dsn)
oracle_cur = oracle_con.cursor()

post_cur.execute("""SELECT *
FROM XXPO_ORDER_CONTROL_PARAM
WHERE LPAD(CAST(facility_code AS VARCHAR), 3, '0') IN (
    SELECT LPAD(CAST(facility_code AS VARCHAR), 3, '0')
    FROM cspoms_replenishment_config
    WHERE POMS_SOURCE = 'CSPOMS'
    AND STATUS = 'Y') """)
data = post_cur.fetchall()

data
# Get column names from the cursor description
column_names = [desc[0] for desc in post_cur.description]
print(column_names)
df = pd.DataFrame(data, columns=column_names)

df

df = df[['facility_code', 'bic_department', 'min_deal', 'max_deal', 'min_additional', 'max_additional', 
                    'min_filler', 'max_filler', 'weight_truck_max', 'cube_truck_max', 'passes', 'p_or_s'
                    ]]
# ,'process_status', 'request_id', 'created_by', 'creation_date', 'last_updated_by', 'last_update_date', 'last_update_login'
# Dictionary with new column names
new_column_names = {'facility_code': "FACILITY",
                    'bic_department': "WAREHOUSE",
                    'min_deal': "DEAL_MINIMUM",
                    'max_deal': "DEAL_MAXIMUM",
                    'min_additional': "ADDITIONAL_MINIMUM",
                    'max_additional': "ADDITIONAL_MAXIMUM",
                    'min_filler': "FILLER_MINIMUM",
                    'max_filler': "FILLER_MAXIMUM",
                    'weight_truck_max': "TRUCK_WEIGHT_MAXIMUM",
                    'cube_truck_max': "TRUCK_CUBE_MAXIMUM",
                    'last_update_date': "SYSDATE",
                    'passes': "DEAL_LOOP_VARIABLE",
                    'p_or_s': "FIRST_INCREMENT_P_OR_S"}

df.rename(columns=new_column_names, inplace=True)
print(df)

# # Get the current system date and time
# current_date = datetime.datetime.now()

# # Format the date according to your requirements
# formatted_date = current_date.strftime("%Y-%m-%d %H:%M:%S")  # Adjust the format as needed

# # Assign the formatted date to the DataFrame column
# df['last_update_date'] = formatted_date
df['request_id'] = -999
df['process_status'] = "N"

# Display the updated DataFrame
df




['facility_code', 'bic_department', 'min_deal', 'max_deal', 'min_additional', 'max_additional', 'min_filler', 'max_filler', 'weight_truck_max', 'cube_truck_max', 'passes', 'p_or_s', 'process_status', 'request_id', 'created_by', 'creation_date', 'last_updated_by', 'last_update_date', 'last_update_login']
  FACILITY WAREHOUSE  DEAL_MINIMUM  DEAL_MAXIMUM  ADDITIONAL_MINIMUM  \
0      304        23             7            13                  14   
1      904        24             7            13                  14   

   ADDITIONAL_MAXIMUM  FILLER_MINIMUM  FILLER_MAXIMUM  TRUCK_WEIGHT_MAXIMUM  \
0                  20              30              50                999999   
1                  20              30              50                999999   

   TRUCK_CUBE_MAXIMUM  DEAL_LOOP_VARIABLE FIRST_INCREMENT_P_OR_S  
0              999999                   1                      P  
1              999999                   1                      P  


,FACILITY,WAREHOUSE,DEAL_MINIMUM,DEAL_MAXIMUM,ADDITIONAL_MINIMUM,ADDITIONAL_MAXIMUM,FILLER_MINIMUM,FILLER_MAXIMUM,TRUCK_WEIGHT_MAXIMUM,TRUCK_CUBE_MAXIMUM,DEAL_LOOP_VARIABLE,FIRST_INCREMENT_P_OR_S,request_id,process_status
0,304,23,7,13,14,20,30,50,999999,999999,1,P,-999,N
1,904,24,7,13,14,20,30,50,999999,999999,1,P,-999,N


In [6]:
df

,FACILITY,WAREHOUSE,DEAL_MINIMUM,DEAL_MAXIMUM,ADDITIONAL_MINIMUM,ADDITIONAL_MAXIMUM,FILLER_MINIMUM,FILLER_MAXIMUM,TRUCK_WEIGHT_MAXIMUM,TRUCK_CUBE_MAXIMUM,DEAL_LOOP_VARIABLE,FIRST_INCREMENT_P_OR_S
0,304,23,7,13,14,20,30,50,999999,999999,1,P
1,904,24,7,13,14,20,30,50,999999,999999,1,P


In [3]:

merge_query = """MERGE INTO P2P_DATA.XXP2P_BCPS2EXA_ORDER_CRTL main
USING (
    SELECT DISTINCT
        FACILITY,
        WAREHOUSE,
        DEAL_MINIMUM,
        DEAL_MAXIMUM,
        ADDITIONAL_MINIMUM,
        ADDITIONAL_MAXIMUM,
        FILLER_MINIMUM,
        FILLER_MAXIMUM,
        TRUCK_WEIGHT_MAXIMUM,
        TRUCK_CUBE_MAXIMUM,
        DEAL_LOOP_VARIABLE,
        FIRST_INCREMENT_P_OR_S,
        PROCESS_STATUS,
        CREATION_DATE,
        CREATED_BY,
        REQUEST_ID
    FROM P2P_DATA.XXP2P_BCPS2EXA_ORDER_CRTL_STG  
    WHERE request_id = -999
) stg
ON (
    main.FACILITY = stg.FACILITY
    AND main.WAREHOUSE = stg.WAREHOUSE
)
WHEN MATCHED THEN
    UPDATE SET 
        main.DEAL_MINIMUM = stg.DEAL_MINIMUM,
        main.DEAL_MAXIMUM = stg.DEAL_MAXIMUM,
        main.ADDITIONAL_MINIMUM = stg.ADDITIONAL_MINIMUM,
        main.ADDITIONAL_MAXIMUM = stg.ADDITIONAL_MAXIMUM,
        main.FILLER_MINIMUM = stg.FILLER_MINIMUM,
        main.FILLER_MAXIMUM = stg.FILLER_MAXIMUM,
        main.TRUCK_WEIGHT_MAXIMUM = stg.TRUCK_WEIGHT_MAXIMUM,
        main.TRUCK_CUBE_MAXIMUM = stg.TRUCK_CUBE_MAXIMUM,
        main.DEAL_LOOP_VARIABLE = stg.DEAL_LOOP_VARIABLE,
        main.FIRST_INCREMENT_P_OR_S = stg.FIRST_INCREMENT_P_OR_S,
        main.REQUEST_ID = stg.REQUEST_ID,
        main.CREATION_DATE = stg.CREATION_DATE,
        main.CREATED_BY = stg.CREATED_BY,
        main.LAST_UPDATE_DATE = SYSDATE,
        main.LAST_UPDATED_BY = -2
WHEN NOT MATCHED THEN
    INSERT (
        FACILITY, 
        WAREHOUSE, 
        DEAL_MINIMUM, 
        DEAL_MAXIMUM, 
        ADDITIONAL_MINIMUM, 
        ADDITIONAL_MAXIMUM, 
        FILLER_MINIMUM, 
        FILLER_MAXIMUM, 
        TRUCK_WEIGHT_MAXIMUM, 
        TRUCK_CUBE_MAXIMUM, 
        DEAL_LOOP_VARIABLE, 
        FIRST_INCREMENT_P_OR_S,
        LAST_UPDATED_BY,
        LAST_UPDATE_DATE,
        CREATION_DATE,
        CREATED_BY,
        REQUEST_ID
    )
    VALUES (
        stg.FACILITY, 
        stg.WAREHOUSE, 
        stg.DEAL_MINIMUM, 
        stg.DEAL_MAXIMUM, 
        stg.ADDITIONAL_MINIMUM, 
        stg.ADDITIONAL_MAXIMUM, 
        stg.FILLER_MINIMUM, 
        stg.FILLER_MAXIMUM, 
        stg.TRUCK_WEIGHT_MAXIMUM, 
        stg.TRUCK_CUBE_MAXIMUM, 
        stg.DEAL_LOOP_VARIABLE, 
        stg.FIRST_INCREMENT_P_OR_S,
        -3,
        SYSDATE,
        stg.CREATION_DATE,
        stg.CREATED_BY,
        stg.REQUEST_ID
    )"""


print(merge_query)
oracle_cur.execute(merge_query)
merge_count = oracle_cur.rowcount
table_main = 'P2P_DATA.XXP2P_BCPS2EXA_ORDER_CRTL'
print(f'MERGED {merge_count} EXCEL RECORDS TO {table_main} TABLE')
oracle_con.commit()

MERGE INTO P2P_DATA.XXP2P_BCPS2EXA_ORDER_CRTL main
USING (
    SELECT DISTINCT
        FACILITY,
        WAREHOUSE,
        DEAL_MINIMUM,
        DEAL_MAXIMUM,
        ADDITIONAL_MINIMUM,
        ADDITIONAL_MAXIMUM,
        FILLER_MINIMUM,
        FILLER_MAXIMUM,
        TRUCK_WEIGHT_MAXIMUM,
        TRUCK_CUBE_MAXIMUM,
        DEAL_LOOP_VARIABLE,
        FIRST_INCREMENT_P_OR_S,
        PROCESS_STATUS,
        CREATION_DATE,
        CREATED_BY,
        REQUEST_ID
    FROM P2P_DATA.XXP2P_BCPS2EXA_ORDER_CRTL_STG  
    WHERE request_id = -999
) stg
ON (
    main.FACILITY = stg.FACILITY
    AND main.WAREHOUSE = stg.WAREHOUSE
)
WHEN MATCHED THEN
    UPDATE SET 
        main.DEAL_MINIMUM = stg.DEAL_MINIMUM,
        main.DEAL_MAXIMUM = stg.DEAL_MAXIMUM,
        main.ADDITIONAL_MINIMUM = stg.ADDITIONAL_MINIMUM,
        main.ADDITIONAL_MAXIMUM = stg.ADDITIONAL_MAXIMUM,
        main.FILLER_MINIMUM = stg.FILLER_MINIMUM,
        main.FILLER_MAXIMUM = stg.FILLER_MAXIMUM,
        main.TRUCK_WEIGHT_M

DatabaseError: ORA-30926: unable to get a stable set of rows in the source tables

In [2]:


table_name = "P2P_DATA.XXP2P_BCPS2EXA_ORDER_CRTL_STG"
fields = [col for col in df.columns]
str_sub = ', '.join([':%d' % (i + 1) for i in range(len(fields))])
insert_query = f"INSERT INTO {table_name} ({', '.join(fields)}) VALUES ({str_sub})"
print(insert_query)
oracle_cur.executemany(insert_query, df.values.tolist())
insert_count = oracle_cur.rowcount
oracle_con.commit()

print(f'Inserted {insert_count} records into {table_name} table.')


INSERT INTO P2P_DATA.XXP2P_BCPS2EXA_ORDER_CRTL_STG (FACILITY, WAREHOUSE, DEAL_MINIMUM, DEAL_MAXIMUM, ADDITIONAL_MINIMUM, ADDITIONAL_MAXIMUM, FILLER_MINIMUM, FILLER_MAXIMUM, TRUCK_WEIGHT_MAXIMUM, TRUCK_CUBE_MAXIMUM, DEAL_LOOP_VARIABLE, FIRST_INCREMENT_P_OR_S, request_id, process_status) VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14)
Inserted 2 records into P2P_DATA.XXP2P_BCPS2EXA_ORDER_CRTL_STG table.


In [5]:
oracle_cur.execute("SELECT * FROM P2P_DATA.XXP2P_BCPS2EXA_ORDER_CRTL_STG")
dff = oracle_cur.fetchall()
dff

[('001',
  None,
  '7',
  '13',
  '14',
  '20',
  '30',
  '70',
  '999999',
  '999999',
  '91721',
  '1',
  'P',
  'P',
  512131,
  datetime.datetime(2022, 10, 7, 15, 20),
  -1,
  datetime.datetime(2022, 10, 21, 16, 19, 21),
  -1),
 ('001',
  '004',
  '7',
  '13',
  '14',
  '20',
  '21',
  '27',
  '46000',
  '3100',
  '50106',
  '1',
  'P',
  'P',
  512131,
  datetime.datetime(2022, 10, 7, 15, 20),
  -1,
  datetime.datetime(2022, 10, 21, 16, 19, 21),
  -1),
 ('001',
  '020',
  '7',
  '13',
  '14',
  '20',
  '40',
  '70',
  '999999',
  '999999',
  '91721',
  '1',
  'P',
  'P',
  512131,
  datetime.datetime(2022, 10, 7, 15, 20),
  -1,
  datetime.datetime(2022, 10, 21, 16, 19, 21),
  -1),
 ('001',
  '021',
  '7',
  '13',
  '14',
  '20',
  '23',
  '56',
  '999999',
  '999999',
  '121021',
  '1',
  'P',
  'P',
  512131,
  datetime.datetime(2022, 10, 7, 15, 20),
  -1,
  datetime.datetime(2022, 10, 21, 16, 19, 21),
  -1),
 ('001',
  '022',
  '7',
  '13',
  '14',
  '20',
  '50',
  '70',
  '999

In [31]:
print(df)
print(df.info())


  T2_TABLE_26_FR_FAC T2_TABLE_26_FR_WH T2_TABLE_26_TO_FAC  \
0                 15                 2                 23   
1                 15                 2                 43   

  T2_TABLE_26_ORD_INCR T2_TABLE_26_VENDOR T2_TABLE_26_MIN_PALLET  \
0                    P             777965                     60   
1                    P             777965                     60   

  T2_TABLE_26_MIN_WGHT T2_TABLE_26_MIN_CUBE T2_TABLE_26_LEAD_TIME  \
0                42000                 1500                     1   
1                42000                 1500                     1   

  T2_TABLE_26_OP_DAYS creation_date created_by    last_update_date  \
0                   3    2024-01-01          1 2024-03-27 13:36:35   
1                   3    2024-01-01          1 2024-03-27 13:36:35   

   request_id process_status     last_updated_date  
0        -999              N  27-Mar-2024 13:38:02  
1        -999              N  27-Mar-2024 13:38:02  
<class 'pandas.core.frame.DataFra

In [30]:
# Convert date columns to datetime format
df['creation_date'] = pd.to_datetime(df['creation_date'])
df['last_update_date'] = pd.to_datetime(df['last_update_date'])
# Convert numeric columns to integer format
df['request_id'] = df['request_id'].astype(int)

In [37]:
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
import datetime
import cx_Oracle

# Database connection parameters
# Replace these with your actual database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()

username = 'ebs_apps'
password = 'ebs_apps'
host = 'csdwddb.cswg.com' #EBSD2
port = '1521'
service_name = 'DWD_USR' #EBSD2

# Create a connection to Oracle
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
oracle_con = cx_Oracle.connect(username, password, dsn)
oracle_cur = oracle_con.cursor()

# Retrieve data from PostgreSQL
post_cur.execute("""SELECT *
FROM XXPO_PULLBAK_PARAM
WHERE LPAD(CAST(to_fac_code AS VARCHAR), 3, '0') IN (
    SELECT LPAD(CAST(facility_code AS VARCHAR), 3, '0')
    FROM cspoms_replenishment_config
    WHERE POMS_SOURCE = 'CSPOMS'
    AND STATUS = 'Y'
)""")
data = post_cur.fetchall()

# Get column names from the cursor description
column_names = [desc[0] for desc in post_cur.description]

# Create a DataFrame from the fetched data
df = pd.DataFrame(data, columns=column_names)

# Select the required columns and rename them
df = df[['from_fac_code', 'from_warehouse_num', 'to_fac_code',
         'incr', 'vendor_number', 'max_pallets_allowed',
         'max_weight_allowed', 'max_cube_allowed', 'lead_time', 'op_days',
         'creation_date', 'created_by']]

# Rename the columns according to the desired format
new_column_names = {'from_fac_code': "T2_TABLE_26_FR_FAC",
                    'from_warehouse_num': "T2_TABLE_26_FR_WH",
                    'to_fac_code': "T2_TABLE_26_TO_FAC",
                    'incr': "T2_TABLE_26_ORD_INCR",
                    'vendor_number': "T2_TABLE_26_VENDOR",
                    'max_pallets_allowed': "T2_TABLE_26_MIN_PALLET",
                    'max_weight_allowed': "T2_TABLE_26_MIN_WGHT",
                    'max_cube_allowed': "T2_TABLE_26_MIN_CUBE",
                    'lead_time': "T2_TABLE_26_LEAD_TIME",
                    'op_days': "T2_TABLE_26_OP_DAYS"}
df.rename(columns=new_column_names, inplace=True)

# Convert date columns to datetime format
df['creation_date'] = pd.to_datetime(df['creation_date'])

# Get the current system date and time
current_date = datetime.datetime.now().date()

# Assign the current date to the last_update_date column
df['last_update_date'] = current_date

# Set the request_id, process_status, and last_updated_date columns
df['request_id'] = -999
df['process_status'] = "N"

# Insert the DataFrame into Oracle
table_name = "P2P_DATA.XXPO_PULLBACK_BICEPS2EXA_STG"
fields = [col for col in df.columns]
str_sub = ', '.join([':%d' % (i + 1) for i in range(len(fields))])
insert_query = f"INSERT INTO {table_name} ({', '.join(fields)}) VALUES ({str_sub})"
oracle_cur.executemany(insert_query, df.values.tolist())
insert_count = oracle_cur.rowcount
oracle_con.commit()
# merge_query = f"""
# select * from XXPO_PULLBACK_BICEPS2EXA_DATA where RECORD_ID IS NULL
# """
# print(merge_query)
# oracle_cur.execute(merge_query)
# count = oracle_cur.rowcount
# print(f'MERGED {count} EXCEL RECORDS TO {table_main} TABLE')

oracle_con.commit()
# Print a confirmation message
print(f'Inserted {insert_count} records into {table_name} table.')


Inserted 2 records into P2P_DATA.XXPO_PULLBACK_BICEPS2EXA_STG table.
